# 事前学習モデルのファインチューニング

事前学習済みモデルを利用することで，計算コストを削減し，CO2排出量を減少させ，ゼロから学習することなく最新のモデルを使用できるという利点があります．
Hugging Face Transformers は，さまざまなタスクに対応した数千もの事前学習済みモデルを提供します．
事前学習済みモデルを使用する場合，特定のタスクにあわせたデータセットで学習します．
これはファインチューニングという呼ばれ，非常に強力な学習技術です．

**※説明の読みやすさのために，警告を非表示にします．実際のプログラムでは警告の表示を推奨します．**

In [1]:
# 警告の表示を停止
import warnings
warnings.filterwarnings('ignore')
# Transformersのログを非表示
from transformers import logging
logging.set_verbosity_error()

## データセットの準備

事前学習済みのモデルをファインチューニングする前に，データセットをダウンロードしてトレーニング用に準備する必要があります．

まず，[Yelp Reviews](https://huggingface.co/datasets/yelp_review_full)データセットを読み込みましょう:

In [2]:
from datasets import load_dataset
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

Hugging Face Datasets の`map`メソッドを利用して，データ全体に前処理（トークナイザによるトークン化，パディング/切り詰め）を行います:

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

必要に応じて，実行時間の短縮のためにフルデータセットの小さなサブセットを作ります:

In [4]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## 学習

実際の学習方法は，利用するフレームワークによって異なります．

## PyTorch Trainer を利用した学習

Hugging Face Transformers は，提供するモデルの学習に最適化された`Trainer`クラスを提供しています．
独自の学習ループを記述することなく，学習を開始しやすくしています．
`Trainer API`は，ログの記録，勾配累積，混合精度など，さまざまな学習オプションと機能をサポートしています．

まず，モデルをロードし，予想されるラベルの数を指定します．
`Yelp Review`の`datset card`から，5つのラベルがあることがわかります:

In [5]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

> 一部の学習済みの重みが使用されず，ランダムに初期化されたという警告が表示されることがありますが，これは正常です．
> BERTモデルの事前学習済みのヘッドは破棄され，ランダムに初期化された分類ヘッドで置き換えられたためです．
> この新しいモデルヘッドをシーケンス分類タスクでファインチューニングするのです．

### 学習用のハイパーパラメータ

次に，学習オプションを有効にするための全てのハイパーパラメータと，調整できるハイパーパラメータをふくむ`TrainingArguments`クラスを作成します．
ここでは，デフォルトの学習ハイパーパラメータを使用しますが，最適な設定を見つけるためにこれらを調整しても構いません．

学習のチェックポイントを保存する場所を指定します:

In [6]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### 評価

`Trainer`は，学習中に自動的にモデルの性能を評価しません．
評価値を計算して報告する関数を`Trainer`に渡す必要があります．
Hugging Face Evaluateライブラリでは，`evaluate.load`関数を使用して読み込むことができるシンプルな`accuracy`関数を提供しています．

In [7]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

`metric`の`~evaluate.compute`を呼び出して，予測の正確度を計算します．
`compute`に予測を渡す前に，予測のロジットに変換する必要があります．
（すべてのHugging Face Transformesモデルはロジットを返します）

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

評価値をファインチューニング中に監視したい場合，学習の引数で`eval_strategy`パラメータを指定して，各エポックの終了時に評価値を報告させます:

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

### Trainer

モデル，学習のための引数，学習およびデータセット，評価関数を使用して`Trainer`オブジェクトを作成します:

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

その後，`train()`を呼び出してファインチューニングを実行します:

In [11]:
trainer.train()

{'eval_loss': 1.0938891172409058, 'eval_accuracy': 0.551, 'eval_runtime': 3.1884, 'eval_samples_per_second': 313.634, 'eval_steps_per_second': 39.204, 'epoch': 1.0}
{'eval_loss': 1.0334631204605103, 'eval_accuracy': 0.541, 'eval_runtime': 3.1999, 'eval_samples_per_second': 312.508, 'eval_steps_per_second': 39.063, 'epoch': 2.0}
{'eval_loss': 0.986595869064331, 'eval_accuracy': 0.601, 'eval_runtime': 3.2094, 'eval_samples_per_second': 311.584, 'eval_steps_per_second': 38.948, 'epoch': 3.0}
{'train_runtime': 41.8487, 'train_samples_per_second': 71.687, 'train_steps_per_second': 8.961, 'train_loss': 1.0666346842447916, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=1.0666346842447916, metrics={'train_runtime': 41.8487, 'train_samples_per_second': 71.687, 'train_steps_per_second': 8.961, 'train_loss': 1.0666346842447916, 'epoch': 3.0})

## Native PyTorch での学習

(割愛)

## 追加情報

- [Hugging Face Transformers Examples](https://github.com/huggingface/transformers/tree/main/examples)には，PyTorchとTensorFlowで一般的なNLPタスクを学習するスクリプトが含まれています．
- [Hugging Face Transformers Notebooks](https://huggingface.co/docs/transformers/ja/notebooks)には，特定のタスクにモデルをファインチューニングする方法に関するさまざまなノートブックが含まれています．